In [1]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json

sys.path.append(str(Path("..").resolve()))
from src import *

# Seeding Missing Collections and Entries
ℹ️ This notebook require the execution of [`1.1-Migrating_MySQL_To_MongoDB`](1.1-Migrating_MySQL_To_MongoDB.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `2_migration_preprocessed` snapshot.

In [2]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/2_migration_preprocessed/omero_museum"

In [3]:
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]:
[artworks]: _id date description room type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[employees]: _id birth_date curriculum date_start email gender hometown name phone_number surname
[limited_events]:
[messages]:
[rooms]:
[suppliers]:
[surveys]: _id accompanying_persons_visit date_of_compilation evaluation_of_experience evaluation_of_facility evaluation_of_visit number_of_visits reason_for_visit return title_of_studies type_of_visit
[tickets]: _id date price
[visitors]: _id cell phone number email fare surname
[workshops]: _id duration price_class price_person type


## Seeding DB


### 1. Defining IDs of documents

In [4]:
ids = {
    "authors": [_["_id"] for _ in db.authors.find({})],
    "surveys": range(100),
    "visitors": range(50),
    "tickets": range(50),
    "workshops": [_["_id"] for _ in db.workshops.find({})],
    "activities": range(25),
    "rooms": [
        "Greco e Romano",
        "Medievale e 400",
        "Ancona",
        "Rinascimentale",
        "Contemporaneo",
        "Impressionismo",
    ],
    "messages": range(100),
    "suppliers": range(25),
    "artworks": [
        "Mona Lisa",
        "The Starry Night",
        "The Persistence of Memory",
        "The Birth of Venus",
        "The Night Watch",
        "Girl with a Pearl Earring",
        "Guernica",
        "American Gothic",
        "The Scream",
        "Les Demoiselles d'Avignon",
        "The Kiss",
        "The Last Supper",
        "Liberty Leading the People",
        "The Great Wave off Kanagawa",
        "Nighthawks",
        "Campbell's Soup Cans",
        "Impression, Sunrise",
        "No. 5, 1948",
        "A Sunday Afternoon on the Island of La Grande Jatte",
        "The School of Athens",
    ],
    "limited_events": [
        "Digital Preservation of Cultural Heritage",
        "Museum Education in the 21st Century",
        "Community Engagement and Inclusion",
        "Sustainability in Museum Practices",
        "Virtual and Augmented Reality in Exhibitions",
        "Ethics of Artifact Repatriation",
        "Artificial Intelligence in Collections Management",
        "Storytelling Through Curatorial Design",
        "Accessibility and Universal Design in Museums",
        "Cross-Cultural Dialogue Through Exhibits",
        "The Future of Archaeological Conservation",
        "Museums as Spaces for Social Justice",
        "Data-Driven Visitor Experience",
        "Collaborative Curation with Indigenous Communities",
        "The Role of Museums in Climate Change Awareness",
        "Gamification of Museum Learning",
        "Blockchain for Provenance Tracking",
        "Digital Twins of Artifacts",
        "Museums and Mental Health",
        "Hybrid Exhibitions: Physical Meets Digital",
    ],
}

### 2. Defining constants

In [5]:
themes = [
    "Preservation",
    "Education",
    "Engagement",
    "Sustainability",
    "Virtuality",
    "Repatriation",
    "AI",
    "Storytelling",
    "Accessibility",
    "Dialogue",
    "Conservation",
    "Justice",
    "Data",
    "Collaboration",
    "Climate",
    "Gamification",
    "Blockchain",
    "DigitalTwins",
    "Wellbeing",
    "Hybridization",
]
surveys = db.surveys.find({}, {"_id": 0}).to_list()
artwork_descriptions = [doc["description"] for doc in db.artworks.find({})]
artwork_types = ["cast", "statue", "mechanical", "painting", "picture", "relief"]
artwork_periods = [
    "Prehistoric",
    "Ancient",
    "Classical",
    "Medieval",
    "Renaissance",
    "Baroque",
    "Industrial",
    "Modern",
]
artwork_techniques = [
    "OilPainting",
    "Fresco",
    "Sculpture",
    "Engraving",
    "Mosaic",
    "Watercolor",
    "Etching",
    "Photography",
    "Installation",
    "DigitalArt",
]
artwork_materials = [
    "Marble",
    "Bronze",
    "Wood",
    "Canvas",
    "Paper",
    "Clay",
    "Glass",
    "Stone",
    "Textile",
    "Steel",
]

### 3. Seed Aggregates

In [6]:
# Aggregates of aggregates
messages_seeder = DocSeeder(
    {
        "customerId": EntrySeeder(range(len(ids["visitors"]))),
        "sentDate": EntrySeeder("past_datetime"),
        "type": EntrySeeder(["text", "audio", "picture"]),
        "content": EntrySeeder("text"),
        "deliveryStatus": EntrySeeder(["sending", "sent", "received", "seen"]),
        "length": EntrySeeder(range(10, 300, 5), p=0.25),
        "uri": EntrySeeder("url", p=0.35),
    }
)
messages = messages_seeder.seed(500)

In [7]:
# Aggregates
aggregate_seeders = {
    "comments": DocSeeder(
        {
            "rating": EntrySeeder(range(1, 6)),
            "date": EntrySeeder("past_datetime"),
            "message": EntrySeeder("text"),
        }
    ),
    "trades": DocSeeder(
        {
            "date_start": EntrySeeder("past_datetime"),
            "date_end": EntrySeeder("past_datetime"),
            "win_price": EntrySeeder(range(1000, 1000000, 10)),
            "winner_id": EntrySeeder(ids["visitors"]),
            "price_max": EntrySeeder(range(1000, 1000000, 10)),
            "price_min": EntrySeeder(range(0, 5000, 5)),
            "price_start": EntrySeeder(range(0, 1000, 5)),
            "min_increase": EntrySeeder(range(0, 100, 5)),
            "is_auction": EntrySeeder("boolean"),
        }
    ),
    "tickets": DocSeeder(
        {
            "price": EntrySeeder([0, 4.5, 6, 0, 4.5, 6, 12, 70]),
            "date": EntrySeeder("past_datetime"),
            "check_in": EntrySeeder("boolean"),
        }
    ),
    "chats": DocSeeder(
        {
            "date_creation": EntrySeeder("past_datetime"),
            "new_messages": EntrySeeder(
                lambda _: _.random_elements(messages, length=10, unique=True)
            ),
            "old_messages": EntrySeeder(
                lambda _: _.random_elements(
                    ids["messages"], length=_.random_int(max=20), unique=True
                )
            ),
        }
    ),
}
comments = list(aggregate_seeders["comments"].seed(500))
trades = list(aggregate_seeders["trades"].seed(100))
tickets = list(aggregate_seeders["tickets"].seed(500))
chats = list(aggregate_seeders["chats"].seed(100))

### 4. Seed Collections

In [8]:
seeders = {
    "rooms": DocSeeder(
        {
            "_id": EntrySeeder(ids["rooms"], unique=True),
            "floor": EntrySeeder(range(5), p=0.6),
        }
    ),
    "activities": DocSeeder(
        {
            "_id": EntrySeeder(ids["activities"], unique=True),
            "room": EntrySeeder(ids["rooms"]),
            "duration": EntrySeeder(range(15, 600, 15)),
            "enrolled": EntrySeeder(range(20)),
            "startDate": EntrySeeder("past_datetime"),
            "capacity": EntrySeeder(range(20, 100)),
            "ticketIds": EntrySeeder(
                lambda _: _.random_elements(
                    range(len(ids["tickets"])),
                    length=_.random_int(max=15),
                    unique=True,
                )
            ),
            "workshopTitle": EntrySeeder(ids["workshops"], p=0.4),
        }
    ),
    "limited_events": DocSeeder(
        {
            "_id": EntrySeeder(ids["limited_events"], unique=True),
            "capacity": EntrySeeder(range(20, 50)),
            "type": EntrySeeder(["exhibition", "conference"]),
            "startDate": EntrySeeder("past_datetime"),
            "endDate": EntrySeeder("past_datetime"),
            "artist": EntrySeeder(range(10)),
            "theme": EntrySeeder(themes, p=0.5),
            "description": EntrySeeder("text"),
            "roomName": EntrySeeder(ids["rooms"]),
            "authorIds": EntrySeeder(
                lambda _: _.random_elements(
                    range(15), length=_.random_int(max=4), unique=True
                )
            ),
            "artworkTitles": EntrySeeder(
                lambda _: _.random_elements(
                    ids["artworks"], length=_.random_int(min=3, max=10), unique=True
                ),
                p=0.8,
            ),
            "ticketIds": EntrySeeder(
                lambda _: _.random_elements(
                    range(len(ids["tickets"])), length=_.random_int(max=10), unique=True
                ),
            ),
        }
    ),
    "messages": DocSeeder(
        {
            "_id": EntrySeeder(ids["messages"], unique=True),
            "customerId": EntrySeeder(range(len(ids["visitors"]))),
            "sentDate": EntrySeeder("past_datetime"),
            "type": EntrySeeder(["text", "audio", "picture"]),
            "content": EntrySeeder("text"),
            "deliveryStatus": EntrySeeder(["sending", "sent", "received", "seen"]),
            "length": EntrySeeder(range(10, 300, 5), p=0.25),
            "uri": EntrySeeder("url", p=0.35),
        }
    ),
    "suppliers": DocSeeder(
        {
            "_id": EntrySeeder(ids["suppliers"], unique=True),
            "name": EntrySeeder("name"),
            "iban": EntrySeeder("iban"),
            "isState": EntrySeeder("boolean", p=0.5),
            "isMuseum": EntrySeeder("boolean"),
            "email": EntrySeeder("email"),
            "phoneNum": EntrySeeder("phone_number"),
            "surveys": EntrySeeder(
                lambda _: _.random_elements(
                    surveys, length=_.random_int(max=15), unique=True
                )
            ),
        }
    ),
    "artworks": DocSeeder(
        {
            "_id": EntrySeeder(ids["artworks"], unique=True),
            "date": EntrySeeder("past_datetime"),
            "type": EntrySeeder(artwork_types),
            "description": EntrySeeder(artwork_descriptions),
            "isOriginal": EntrySeeder("boolean"),
            "size": EntrySeeder(range(10, 500, 10)),
            "period": EntrySeeder(artwork_periods),
            "sellerId": EntrySeeder(ids["suppliers"], p=0.35),
            "donatorId": EntrySeeder(ids["visitors"], p=0.25),
            "donationState": EntrySeeder(
                ["in_progress", "accepted", "refused"], p=0.25
            ),
            "locationName": EntrySeeder("city"),
            "authorIds": EntrySeeder(
                lambda _: _.random_elements(ids["authors"], length=_.random_int(max=3))
            ),
            "tecniques": EntrySeeder(
                lambda _: _.random_elements(
                    artwork_techniques, length=_.random_int(max=3)
                )
            ),
            "materials": EntrySeeder(
                lambda _: _.random_elements(
                    artwork_materials, length=_.random_int(max=5)
                )
            ),
            "comments_star_1": EntrySeeder(
                lambda _: _.random_elements(comments, length=_.random_int(max=5))
            ),
            "comments_star_2": EntrySeeder(
                lambda _: _.random_elements(comments, length=_.random_int(max=5))
            ),
            "comments_star_3": EntrySeeder(
                lambda _: _.random_elements(comments, length=_.random_int(max=5))
            ),
            "comments_star_4": EntrySeeder(
                lambda _: _.random_elements(comments, length=_.random_int(max=5))
            ),
            "comments_star_5": EntrySeeder(
                lambda _: _.random_elements(comments, length=_.random_int(max=5))
            ),
            "trade": EntrySeeder(trades),
        }
    ),
    "visitors": DocSeeder(
        {
            "_id": EntrySeeder(ids["visitors"], unique=True),
            "isCustomer": EntrySeeder("boolean"),
            "birthDate": EntrySeeder("past_datetime"),
            "impairment": EntrySeeder(["deaf", "blind"], p=0.75),
            "gender": EntrySeeder(["male", "female"]),
            "land": EntrySeeder("country"),
            "tickets": EntrySeeder(
                lambda _: _.random_elements(tickets, length=_.random_int(max=4))
            ),
            "surveys": EntrySeeder(
                lambda _: _.random_elements(surveys, length=_.random_int(max=3))
            ),
            "surname": EntrySeeder("last_name"),
            "name": EntrySeeder("first_name"),
            "phoneNum": EntrySeeder("phone_number"),
            "email": EntrySeeder("email"),
            "chat": EntrySeeder(chats, p=0.5),
            "comments": EntrySeeder(
                lambda _: _.random_elements(comments, length=_.random_int(max=3)), p=0.5
            ),
            "donationIds": EntrySeeder(
                lambda _: _.random_elements(
                    ids["artworks"], length=_.random_int(max=3)
                ),
                p=0.25,
            ),
            "saleIds": EntrySeeder(
                lambda _: _.random_elements(
                    ids["artworks"], length=_.random_int(max=3)
                ),
                p=0.25,
            ),
            "tradeIds": EntrySeeder(
                lambda _: _.random_elements(
                    ids["artworks"], length=_.random_int(max=3)
                ),
                p=0.5,
            ),
        }
    ),
}

In [9]:
# Seed!
for coll, seeder in seeders.items():
    cprint("Seeding", f"green:{coll}", "...")
    db[coll].delete_many({})
    db[coll].insert_many(seeder.seed(len(ids[coll])))

Seeding rooms ...
Seeding activities ...
Seeding limited_events ...
Seeding messages ...
Seeding suppliers ...
Seeding artworks ...
Seeding visitors ...


### 5. Remove Unneeded Collections

In [10]:
for coll in ["surveys", "tickets"]:
    cprint("Dropping", f"red:{coll}", "...")
    db.drop_collection(coll)

Dropping surveys ...
Dropping tickets ...


From the migration, the following collections were retained
- departments
- employees
- authors
- workshops
- surveys

## Dump Final Database

In [11]:
%%capture
!mongodump --host localhost:27017 --db omero_museum --out "../backup/3_seeded"